## Busca de Imóveis

Utilizar sites de corretoras para criar um dataset de imóveis de Curitiba

In [27]:
from bs4 import BeautifulSoup, Comment
import requests
import re
from time import sleep
import csv

SITE = "https://www.zapimoveis.com.br/venda/imoveis/pr+curitiba/?onde=,Paran%C3%A1,Curitiba,,,,,city,BR%3EParana%3ENULL%3ECuritiba,-25.437238,-49.269973,&pagina={pagina}&tipo=Im%C3%B3vel%20usado&transacao=Venda"

#### Funções

In [28]:
def list_to_csv(list,file):
    with open(file, 'w',newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['local','preco','metragem','quartos','garagens','banheiros'])
        writer.writerows(list)

In [29]:
def get_page(s,p):
    headers = requests.utils.default_headers()
    headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })

    link = requests.get(s.format(pagina=p), headers=headers)

    if link.ok:

        result = re.sub(r'<!.*?->','', link.content.decode("utf-8"))
        result = result.replace('\n','')

        reddit1Content =BeautifulSoup(result,"html.parser")

        cards = reddit1Content.find_all('div', class_='simple-card__box')

        imoveis = []
        for card in cards:
            imovel = []

            local = card.find('h2').text
            imovel.append(local.replace('            ','').replace('          ',''))

            preco =card.find('p').find('strong').text
            imovel.append(preco.replace('R$','').replace(' ','').replace('.',''))

            try:
                metragem = card.find("span", itemprop="floorSize").text
                imovel.append(metragem.replace('m²','').replace(' ',''))
            except:
                imovel.append('')

            try:
                quartos = card.find("span", itemprop="numberOfRooms").text
                imovel.append(quartos.replace(' ',''))
            except:
                imovel.append('')

            try:
                garagens = card.find("li", class_="feature__item text-small js-parking-spaces")
                garagens = garagens.find('span', class_="").text
                imovel.append(garagens.replace(' ',''))
            except:
                imovel.append('')

            try:
                banheiros = card.find("span", itemprop="numberOfBathroomsTotal").text
                imovel.append(banheiros.replace(' ',''))
            except:
                imovel.append('')

            imoveis.append(imovel)
        
    else:
        return False

    return imoveis

#### Execução

In [30]:
imoveis = []

paginas = range(1,20)

for p in paginas:
    i = get_page(SITE,p)
    
    if i == False:
        break
    
    imoveis += i
    sleep(10)

list_to_csv(imoveis,'imoveis.csv')